In [54]:
import os
from river import datasets
from river import ensemble
from confluent_kafka import Producer,Consumer
import certifi
import time
import json

In [55]:
user= os.environ['kafka_username']
password= os.environ['kafka_password']
bsts= os.environ['kafka_bootstrap_servers']

In [56]:
feature_topic = 't_8'

In [57]:
conf = {'bootstrap.servers': bsts,
            'sasl.mechanism': 'PLAIN',
            'security.protocol': 'SASL_SSL',
            'ssl.ca.location': certifi.where(),
            'sasl.username': user,
            'sasl.password': password,
            'batch.num.messages': 2048,
            #'queue.buffering.max.messages': 100,
            'linger.ms': 100,
            'client.id': 'producer-icde-2023'}
producer = Producer(conf)    

In [58]:
max_size=101000
dataset = datasets.MaliciousURL()
data = dataset.take(max_size)

In [59]:
end=0
cnt = 0
st = time.time()
abs_st = time.time()
for f, y in data:
    cnt = cnt + 1    
    d = {}
    d['f']=f
    d['y']=str(y).lower()
    d['st']=time.time()  
            
    v= json.dumps(d).encode('utf-8')
    try:
        producer.produce(feature_topic, value=v, key=str(cnt))
    except:
      print(f'Queue full, flushing {cnt}')
      producer.flush()
      end = time.time()
      print(f'flushing count - {cnt}, time taken in seconds- {end-st} ')        
        
      producer.produce(feature_topic, value=v, key=str(cnt))
 
producer.flush()
end = time.time()
print(f'final flushing count - {cnt}, time taken in seconds- {end-abs_st} ')        

Queue full, flushing 100001
flushing count - 100001, time taken in seconds- 10.381866693496704 
final flushing count - 101000, time taken in seconds- 10.659695386886597 
